# Notebook 4: Transport Mechanisms - HTTP & SSE

## Best Practices for Agentic AI using MCP

---

**Series:** MCP Server Best Practices - Agentic AI Workflows  
**Notebook:** 4 of 13  
**Level:** Intermediate  
**Duration:** ~60 minutes

---

## Learning Objectives

By the end of this notebook, you will:

1. Understand HTTP+SSE transport for remote MCP servers
2. Learn Server-Sent Events (SSE) for streaming responses
3. Build a remote MCP server with proper session management
4. Implement authentication for remote servers
5. Handle connection lifecycle and reconnection
6. Compare STDIO vs HTTP transport for different use cases

## 1. HTTP+SSE Transport Overview

### Why HTTP+SSE?

While STDIO is perfect for local servers, **HTTP+SSE** enables:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    HTTP+SSE TRANSPORT BENEFITS                           │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   🌐 REMOTE ACCESS              👥 MULTI-CLIENT                         │
│   ════════════════              ═════════════                           │
│   • Server on cloud             • Multiple users                        │
│   • Cross-machine               • Concurrent sessions                   │
│   • Mobile clients              • Shared resources                      │
│                                                                          │
│   📈 SCALABILITY                🔐 AUTHENTICATION                       │
│   ═════════════                 ════════════════                        │
│   • Load balancing              • OAuth 2.1 support                     │
│   • Horizontal scaling          • Token-based auth                      │
│   • Cloud deployment            • Fine-grained access                   │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### Architecture Comparison

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    STDIO vs HTTP+SSE                                     │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   STDIO                              HTTP + SSE                         │
│   ═════                              ══════════                         │
│                                                                          │
│   ┌────────┐      stdin/stdout      ┌────────┐                         │
│   │ Client │◄─────────────────────►│ Server │                         │
│   └────────┘      (subprocess)      └────────┘                         │
│                                                                          │
│   • Same machine                                                        │
│   • Single user                                                         │
│   • No auth needed                                                      │
│                                                                          │
│   ════════════════════════════════════════════════════════════════     │
│                                                                          │
│   ┌────────┐      HTTP POST         ┌────────┐                         │
│   │ Client │─────────────────────►│ Server │                         │
│   │   1    │                        │        │                         │
│   └────────┘◄─────────────────────│        │  SSE Stream              │
│                                     │        │                         │
│   ┌────────┐      HTTP POST         │        │                         │
│   │ Client │─────────────────────►│        │                         │
│   │   2    │                        │        │                         │
│   └────────┘◄─────────────────────│        │  SSE Stream              │
│                   Network           └────────┘                         │
│                                                                          │
│   • Remote/cloud                                                        │
│   • Multi-user                                                          │
│   • Auth required                                                       │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Setup for this notebook
import json
from typing import Dict, List, Any, Optional
from dataclasses import dataclass
from datetime import datetime

def pprint(obj: dict, title: str = None):
    if title:
        print(f"\n{'='*60}")
        print(f" {title}")
        print(f"{'='*60}")
    print(json.dumps(obj, indent=2, default=str))

## 2. Understanding Server-Sent Events (SSE)

### What is SSE?

**Server-Sent Events** is a standard for servers to push updates to clients:

| Aspect | Description |
|--------|-------------|
| Protocol | HTTP-based, one-way (server → client) |
| Format | Text stream with `data:`, `event:`, `id:` fields |
| Connection | Long-lived, auto-reconnect |
| Use Case | Real-time updates, streaming responses |

### SSE Message Format

```
event: message
data: {"jsonrpc": "2.0", "id": 1, "result": {...}}

event: message  
data: {"jsonrpc": "2.0", "method": "notifications/resources/updated", ...}

```

In [ ]:
# SSE Message structure
sse_examples = {
    "simple_message": """
event: message
data: {"jsonrpc": "2.0", "id": 1, "result": {"tools": [...]}}

""",
    "notification": """
event: message
data: {"jsonrpc": "2.0", "method": "notifications/tools/list_changed"}

""",
    "endpoint_event": """
event: endpoint
data: /mcp/messages?session_id=abc123

"""
}

print("SSE Message Examples:")
print("=" * 60)
for name, example in sse_examples.items():
    print(f"\n{name}:")
    print(example)

## 3. HTTP+SSE Connection Flow

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    MCP HTTP+SSE CONNECTION FLOW                          │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   CLIENT                                              SERVER            │
│                                                                          │
│      │  1. GET /sse (Accept: text/event-stream)          │              │
│      │ ─────────────────────────────────────────────────►│              │
│      │                                                   │              │
│      │  2. SSE: event=endpoint, data=/messages?sid=xyz   │              │
│      │ ◄─────────────────────────────────────────────────│              │
│      │                                                   │              │
│      │  3. POST /messages?sid=xyz (initialize)           │              │
│      │ ─────────────────────────────────────────────────►│              │
│      │                                                   │              │
│      │  4. SSE: event=message, data={init response}      │              │
│      │ ◄─────────────────────────────────────────────────│              │
│      │                                                   │              │
│      │  5. POST /messages?sid=xyz (initialized notif)    │              │
│      │ ─────────────────────────────────────────────────►│              │
│      │                                                   │              │
│      │         ╔════════════════════════════╗           │              │
│      │         ║   SESSION ESTABLISHED      ║           │              │
│      │         ╚════════════════════════════╝           │              │
│      │                                                   │              │
│      │  6. POST /messages?sid=xyz (tools/call)           │              │
│      │ ─────────────────────────────────────────────────►│              │
│      │                                                   │              │
│      │  7. SSE: event=message, data={tool result}        │              │
│      │ ◄─────────────────────────────────────────────────│              │
│      │                                                   │              │
│      │  8. Server can push notifications anytime         │              │
│      │ ◄─────────────────────────────────────────────────│              │
│      │                                                   │              │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### Key Endpoints

| Endpoint | Method | Purpose |
|----------|--------|----------|
| `/sse` | GET | Establish SSE connection, receive session ID |
| `/messages` | POST | Send JSON-RPC requests |
| `/health` | GET | Health check (optional) |

In [ ]:
# HTTP+SSE endpoints structure
http_endpoints = {
    "/sse": {
        "method": "GET",
        "headers": {
            "Accept": "text/event-stream",
            "Authorization": "Bearer <token>"  # If auth enabled
        },
        "response": "SSE stream with endpoint event containing session URL"
    },
    "/messages": {
        "method": "POST",
        "query_params": {"session_id": "abc123"},
        "headers": {
            "Content-Type": "application/json",
            "Authorization": "Bearer <token>"
        },
        "body": "JSON-RPC request",
        "response": "202 Accepted (response via SSE)"
    },
    "/health": {
        "method": "GET",
        "response": {"status": "healthy", "version": "1.0.0"}
    }
}

pprint(http_endpoints, "HTTP+SSE ENDPOINTS")

## 4. Building an HTTP+SSE MCP Server

### Using FastMCP with HTTP Transport

In [ ]:
# FastMCP HTTP Server Example
FASTMCP_HTTP_SERVER = '''
#!/usr/bin/env python3
"""FastMCP HTTP+SSE Server Example

Run: python http_server.py
Access: http://localhost:8000/sse
"""

from fastmcp import FastMCP
import httpx
from datetime import datetime

# Create MCP server
mcp = FastMCP("RemoteToolServer")


@mcp.tool()
def get_weather(city: str) -> dict:
    """Get current weather for a city.
    
    Args:
        city: City name (e.g., "New York")
    
    Returns:
        Weather data including temperature and conditions
    """
    # Simulated weather data
    return {
        "city": city,
        "temperature": 72,
        "conditions": "Sunny",
        "humidity": 45,
        "timestamp": datetime.now().isoformat()
    }


@mcp.tool()
def search_database(query: str, limit: int = 10) -> list:
    """Search the database with a query.
    
    Args:
        query: Search query string
        limit: Maximum results to return (default 10)
    
    Returns:
        List of matching records
    """
    # Simulated database search
    return [
        {"id": i, "title": f"Result {i} for \'{query}\'", "score": 0.9 - i*0.1}
        for i in range(min(limit, 5))
    ]


@mcp.tool()
async def fetch_url(url: str) -> str:
    """Fetch content from a URL.
    
    Args:
        url: URL to fetch
    
    Returns:
        Response content (first 1000 chars)
    """
    async with httpx.AsyncClient() as client:
        response = await client.get(url, follow_redirects=True)
        return response.text[:1000]


if __name__ == "__main__":
    # Run with HTTP+SSE transport
    mcp.run(
        transport="sse",
        host="0.0.0.0",  # Listen on all interfaces
        port=8000
    )
'''

print("FastMCP HTTP+SSE Server:")
print("=" * 60)
print(FASTMCP_HTTP_SERVER)

### Manual HTTP+SSE Server (Understanding the Protocol)

In [ ]:
# Manual HTTP+SSE Server for understanding
MANUAL_HTTP_SERVER = '''
#!/usr/bin/env python3
"""Manual HTTP+SSE MCP Server - Educational Example

This shows the underlying mechanics. Use FastMCP in production.
"""

import asyncio
import json
import uuid
from aiohttp import web
from typing import Dict
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Store active sessions
sessions: Dict[str, asyncio.Queue] = {}


async def sse_handler(request: web.Request) -> web.StreamResponse:
    """Handle SSE connection - client subscribes here."""
    
    # Create session
    session_id = str(uuid.uuid4())
    sessions[session_id] = asyncio.Queue()
    
    logger.info(f"New SSE connection: {session_id}")
    
    # Setup SSE response
    response = web.StreamResponse(
        status=200,
        headers={
            "Content-Type": "text/event-stream",
            "Cache-Control": "no-cache",
            "Connection": "keep-alive",
            "Access-Control-Allow-Origin": "*"
        }
    )
    await response.prepare(request)
    
    # Send endpoint event with session URL
    endpoint_msg = f"event: endpoint\\ndata: /messages?session_id={session_id}\\n\\n"
    await response.write(endpoint_msg.encode())
    
    try:
        # Keep connection alive, send messages from queue
        while True:
            try:
                message = await asyncio.wait_for(
                    sessions[session_id].get(), 
                    timeout=30.0
                )
                sse_msg = f"event: message\\ndata: {json.dumps(message)}\\n\\n"
                await response.write(sse_msg.encode())
            except asyncio.TimeoutError:
                # Send keepalive
                await response.write(b": keepalive\\n\\n")
    except asyncio.CancelledError:
        pass
    finally:
        del sessions[session_id]
        logger.info(f"SSE connection closed: {session_id}")
    
    return response


async def messages_handler(request: web.Request) -> web.Response:
    """Handle JSON-RPC messages from client."""
    
    session_id = request.query.get("session_id")
    if not session_id or session_id not in sessions:
        return web.json_response(
            {"error": "Invalid session"}, 
            status=400
        )
    
    body = await request.json()
    logger.info(f"Received: {body.get('method')}")
    
    # Process request and queue response
    response = await process_request(body)
    if response:  # Don't queue for notifications
        await sessions[session_id].put(response)
    
    return web.Response(status=202)  # Accepted


async def process_request(request: dict) -> dict:
    """Process JSON-RPC request."""
    method = request.get("method")
    request_id = request.get("id")
    
    if method == "initialize":
        return {
            "jsonrpc": "2.0",
            "id": request_id,
            "result": {
                "protocolVersion": "2025-06-18",
                "capabilities": {"tools": {}},
                "serverInfo": {"name": "HTTPServer", "version": "1.0"}
            }
        }
    elif method == "initialized":
        return None  # Notification, no response
    elif method == "tools/list":
        return {
            "jsonrpc": "2.0",
            "id": request_id,
            "result": {
                "tools": [
                    {
                        "name": "echo",
                        "description": "Echo back the input",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "message": {"type": "string"}
                            },
                            "required": ["message"]
                        }
                    }
                ]
            }
        }
    elif method == "tools/call":
        params = request.get("params", {})
        tool_name = params.get("name")
        args = params.get("arguments", {})
        
        if tool_name == "echo":
            result = f"Echo: {args.get('message', '')}"
        else:
            result = f"Unknown tool: {tool_name}"
        
        return {
            "jsonrpc": "2.0",
            "id": request_id,
            "result": {
                "content": [{"type": "text", "text": result}]
            }
        }
    
    return {
        "jsonrpc": "2.0",
        "id": request_id,
        "error": {"code": -32601, "message": f"Unknown method: {method}"}
    }


app = web.Application()
app.router.add_get("/sse", sse_handler)
app.router.add_post("/messages", messages_handler)

if __name__ == "__main__":
    web.run_app(app, host="0.0.0.0", port=8000)
'''

print("Manual HTTP+SSE Server (Educational):")
print("=" * 60)
print(MANUAL_HTTP_SERVER)

## 5. Session Management

HTTP+SSE requires careful session management:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    SESSION MANAGEMENT                                    │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   SESSION LIFECYCLE                                                     │
│   ═════════════════                                                     │
│                                                                          │
│   1. Client connects to /sse                                            │
│      └─► Server creates session_id, stores Queue                        │
│                                                                          │
│   2. Server sends endpoint event with session URL                       │
│      └─► Client knows where to POST messages                            │
│                                                                          │
│   3. Client sends messages to /messages?session_id=xyz                  │
│      └─► Server validates session, processes, queues response           │
│                                                                          │
│   4. Response delivered via SSE stream                                  │
│      └─► Same session_id ensures correct routing                        │
│                                                                          │
│   5. Connection closes or times out                                     │
│      └─► Server cleans up session state                                 │
│                                                                          │
│   ─────────────────────────────────────────────────────────────────     │
│                                                                          │
│   BEST PRACTICES                                                        │
│   ══════════════                                                        │
│                                                                          │
│   ✅ Use UUIDs for session IDs (unpredictable)                          │
│   ✅ Validate session_id on every request                               │
│   ✅ Implement session timeout (e.g., 30 minutes idle)                  │
│   ✅ Clean up resources when session ends                               │
│   ✅ Handle reconnection gracefully                                     │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Session management implementation
SESSION_MANAGER = '''
import asyncio
import uuid
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from typing import Dict, Optional

@dataclass
class Session:
    """MCP Session state."""
    id: str
    created_at: datetime
    last_activity: datetime
    message_queue: asyncio.Queue = field(default_factory=asyncio.Queue)
    user_id: Optional[str] = None  # For authenticated sessions
    metadata: Dict = field(default_factory=dict)
    
    def is_expired(self, timeout_minutes: int = 30) -> bool:
        """Check if session has expired."""
        return datetime.now() - self.last_activity > timedelta(minutes=timeout_minutes)
    
    def touch(self):
        """Update last activity time."""
        self.last_activity = datetime.now()


class SessionManager:
    """Manage MCP sessions."""
    
    def __init__(self, timeout_minutes: int = 30):
        self.sessions: Dict[str, Session] = {}
        self.timeout_minutes = timeout_minutes
        self._cleanup_task: Optional[asyncio.Task] = None
    
    def create_session(self, user_id: Optional[str] = None) -> Session:
        """Create a new session."""
        session_id = str(uuid.uuid4())
        now = datetime.now()
        session = Session(
            id=session_id,
            created_at=now,
            last_activity=now,
            user_id=user_id
        )
        self.sessions[session_id] = session
        return session
    
    def get_session(self, session_id: str) -> Optional[Session]:
        """Get session by ID, returns None if not found or expired."""
        session = self.sessions.get(session_id)
        if session and not session.is_expired(self.timeout_minutes):
            session.touch()
            return session
        return None
    
    def end_session(self, session_id: str):
        """End and cleanup a session."""
        if session_id in self.sessions:
            del self.sessions[session_id]
    
    async def cleanup_expired(self):
        """Remove expired sessions periodically."""
        while True:
            await asyncio.sleep(60)  # Check every minute
            expired = [
                sid for sid, session in self.sessions.items()
                if session.is_expired(self.timeout_minutes)
            ]
            for sid in expired:
                self.end_session(sid)
    
    def start_cleanup_task(self):
        """Start background cleanup task."""
        self._cleanup_task = asyncio.create_task(self.cleanup_expired())
'''

print("Session Manager Implementation:")
print("=" * 60)
print(SESSION_MANAGER)

## 6. Authentication for HTTP Servers

### OAuth 2.1 Flow (Recommended)

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    OAUTH 2.1 + PKCE FLOW                                 │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   CLIENT                 AUTH SERVER              MCP SERVER            │
│                                                                          │
│      │  1. Generate code_verifier & code_challenge    │                 │
│      │                                                 │                 │
│      │  2. Redirect to /authorize                      │                 │
│      │     ?client_id=...                              │                 │
│      │     &code_challenge=...                         │                 │
│      │     &scope=mcp:tools:read mcp:tools:execute     │                 │
│      │ ───────────────────────────────────────────────►│                 │
│      │                                                 │                 │
│      │  3. User authenticates, grants consent          │                 │
│      │                                                 │                 │
│      │  4. Redirect back with auth code                │                 │
│      │ ◄───────────────────────────────────────────────│                 │
│      │                                                 │                 │
│      │  5. POST /token                                 │                 │
│      │     code=...&code_verifier=...                  │                 │
│      │ ───────────────────────────────────────────────►│                 │
│      │                                                 │                 │
│      │  6. Access token + refresh token                │                 │
│      │ ◄───────────────────────────────────────────────│                 │
│      │                                                 │                 │
│      │  7. GET /sse with Authorization: Bearer token   │                 │
│      │ ─────────────────────────────────────────────────────────────────►│
│      │                                                 │                 │
│      │  8. Server validates token, establishes session │                 │
│      │ ◄─────────────────────────────────────────────────────────────────│
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# OAuth authentication middleware
AUTH_MIDDLEWARE = '''
import jwt
from aiohttp import web
from functools import wraps
from typing import Optional, Set

# MCP-specific OAuth scopes
MCP_SCOPES = {
    "mcp:tools:list",      # List available tools
    "mcp:tools:execute",   # Execute tools
    "mcp:resources:read",  # Read resources
    "mcp:prompts:list",    # List prompts
    "mcp:admin"            # Administrative access
}


class AuthMiddleware:
    """OAuth 2.1 authentication middleware for MCP."""
    
    def __init__(self, jwt_secret: str, required_scopes: Optional[Set[str]] = None):
        self.jwt_secret = jwt_secret
        self.required_scopes = required_scopes or set()
    
    def extract_token(self, request: web.Request) -> Optional[str]:
        """Extract Bearer token from Authorization header."""
        auth_header = request.headers.get("Authorization", "")
        if auth_header.startswith("Bearer "):
            return auth_header[7:]
        return None
    
    def validate_token(self, token: str) -> Optional[dict]:
        """Validate JWT token and return claims."""
        try:
            claims = jwt.decode(
                token,
                self.jwt_secret,
                algorithms=["HS256"],
                options={"require": ["exp", "sub", "scope"]}
            )
            return claims
        except jwt.ExpiredSignatureError:
            return None
        except jwt.InvalidTokenError:
            return None
    
    def check_scopes(self, claims: dict, required: Set[str]) -> bool:
        """Check if token has required scopes."""
        token_scopes = set(claims.get("scope", "").split())
        return required.issubset(token_scopes)
    
    @web.middleware
    async def middleware(self, request: web.Request, handler):
        """Authentication middleware."""
        
        # Skip auth for health check
        if request.path == "/health":
            return await handler(request)
        
        token = self.extract_token(request)
        if not token:
            return web.json_response(
                {"error": "Missing authorization token"},
                status=401
            )
        
        claims = self.validate_token(token)
        if not claims:
            return web.json_response(
                {"error": "Invalid or expired token"},
                status=401
            )
        
        if not self.check_scopes(claims, self.required_scopes):
            return web.json_response(
                {"error": "Insufficient permissions"},
                status=403
            )
        
        # Attach user info to request
        request["user"] = claims
        
        return await handler(request)


# Usage:
# auth = AuthMiddleware(
#     jwt_secret="your-secret",
#     required_scopes={"mcp:tools:list", "mcp:tools:execute"}
# )
# app = web.Application(middlewares=[auth.middleware])
'''

print("OAuth Authentication Middleware:")
print("=" * 60)
print(AUTH_MIDDLEWARE)

## 7. Client-Side Implementation

### Connecting to HTTP+SSE Server

In [ ]:
# HTTP+SSE Client Example
HTTP_CLIENT = '''
#!/usr/bin/env python3
"""HTTP+SSE MCP Client Example"""

import asyncio
import json
import httpx
from typing import AsyncIterator, Optional


class MCPHttpClient:
    """Client for HTTP+SSE MCP servers."""
    
    def __init__(self, base_url: str, token: Optional[str] = None):
        self.base_url = base_url.rstrip("/")
        self.token = token
        self.session_url: Optional[str] = None
        self._request_id = 0
    
    def _headers(self) -> dict:
        headers = {"Content-Type": "application/json"}
        if self.token:
            headers["Authorization"] = f"Bearer {self.token}"
        return headers
    
    def _next_id(self) -> int:
        self._request_id += 1
        return self._request_id
    
    async def connect(self) -> AsyncIterator[dict]:
        """Connect to SSE endpoint and yield messages."""
        async with httpx.AsyncClient() as client:
            async with client.stream(
                "GET",
                f"{self.base_url}/sse",
                headers={**self._headers(), "Accept": "text/event-stream"},
                timeout=None
            ) as response:
                async for line in response.aiter_lines():
                    if line.startswith("event: "):
                        event_type = line[7:]
                    elif line.startswith("data: "):
                        data = line[6:]
                        if event_type == "endpoint":
                            self.session_url = f"{self.base_url}{data}"
                            print(f"Session URL: {self.session_url}")
                        elif event_type == "message":
                            yield json.loads(data)
    
    async def send_request(self, method: str, params: dict = None) -> None:
        """Send JSON-RPC request to server."""
        if not self.session_url:
            raise RuntimeError("Not connected")
        
        request = {
            "jsonrpc": "2.0",
            "id": self._next_id(),
            "method": method,
            "params": params or {}
        }
        
        async with httpx.AsyncClient() as client:
            await client.post(
                self.session_url,
                json=request,
                headers=self._headers()
            )
    
    async def send_notification(self, method: str, params: dict = None) -> None:
        """Send notification (no response expected)."""
        if not self.session_url:
            raise RuntimeError("Not connected")
        
        notification = {
            "jsonrpc": "2.0",
            "method": method,
            "params": params or {}
        }
        
        async with httpx.AsyncClient() as client:
            await client.post(
                self.session_url,
                json=notification,
                headers=self._headers()
            )


async def main():
    """Example usage."""
    client = MCPHttpClient("http://localhost:8000")
    
    # Start receiving messages
    async for message in client.connect():
        print(f"Received: {message}")
        
        # After first message (usually after endpoint event)
        if client.session_url and client._request_id == 0:
            # Initialize
            await client.send_request("initialize", {
                "protocolVersion": "2025-06-18",
                "capabilities": {},
                "clientInfo": {"name": "TestClient", "version": "1.0"}
            })


if __name__ == "__main__":
    asyncio.run(main())
'''

print("HTTP+SSE Client Implementation:")
print("=" * 60)
print(HTTP_CLIENT)

## 8. Handling Reconnection

SSE connections can drop. Handle reconnection gracefully:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    RECONNECTION STRATEGY                                 │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   1. Connection Lost                                                    │
│      └─► Client detects disconnect                                      │
│                                                                          │
│   2. Exponential Backoff                                                │
│      └─► Wait: 1s, 2s, 4s, 8s... (max 30s)                             │
│                                                                          │
│   3. Reconnect to /sse                                                  │
│      └─► New session_id assigned                                        │
│                                                                          │
│   4. Re-initialize                                                      │
│      └─► Send initialize request again                                  │
│                                                                          │
│   5. Restore State (if needed)                                          │
│      └─► Re-subscribe to resources                                      │
│                                                                          │
│   ─────────────────────────────────────────────────────────────────     │
│                                                                          │
│   RETRY POLICY                                                          │
│   ════════════                                                          │
│                                                                          │
│   attempt  │  delay  │  total_wait                                      │
│   ─────────┼─────────┼────────────                                      │
│      1     │   1s    │     1s                                           │
│      2     │   2s    │     3s                                           │
│      3     │   4s    │     7s                                           │
│      4     │   8s    │    15s                                           │
│      5     │  16s    │    31s                                           │
│      6+    │  30s    │   max                                            │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Reconnection handler
RECONNECTION_HANDLER = '''
import asyncio
import random
from typing import Callable, Awaitable


class ReconnectionHandler:
    """Handle automatic reconnection with exponential backoff."""
    
    def __init__(
        self,
        initial_delay: float = 1.0,
        max_delay: float = 30.0,
        max_attempts: int = 10,
        jitter: float = 0.1
    ):
        self.initial_delay = initial_delay
        self.max_delay = max_delay
        self.max_attempts = max_attempts
        self.jitter = jitter
        self.attempt = 0
    
    def reset(self):
        """Reset attempt counter on successful connection."""
        self.attempt = 0
    
    def get_delay(self) -> float:
        """Calculate delay for next reconnection attempt."""
        delay = min(
            self.initial_delay * (2 ** self.attempt),
            self.max_delay
        )
        # Add jitter to prevent thundering herd
        jitter_amount = delay * self.jitter * random.random()
        return delay + jitter_amount
    
    async def execute_with_retry(
        self,
        connect_func: Callable[[], Awaitable],
        on_connected: Callable[[], Awaitable] = None,
        on_disconnected: Callable[[], Awaitable] = None
    ):
        """Execute connection function with automatic retry."""
        while self.attempt < self.max_attempts:
            try:
                self.reset()
                if on_connected:
                    await on_connected()
                await connect_func()
            except Exception as e:
                self.attempt += 1
                if on_disconnected:
                    await on_disconnected()
                
                if self.attempt >= self.max_attempts:
                    raise RuntimeError(f"Max reconnection attempts reached: {e}")
                
                delay = self.get_delay()
                print(f"Connection lost. Reconnecting in {delay:.1f}s (attempt {self.attempt})")
                await asyncio.sleep(delay)


# Usage example:
# handler = ReconnectionHandler()
# await handler.execute_with_retry(
#     connect_func=client.connect,
#     on_connected=lambda: print("Connected!"),
#     on_disconnected=lambda: print("Disconnected!")
# )
'''

print("Reconnection Handler:")
print("=" * 60)
print(RECONNECTION_HANDLER)

## 9. Transport Comparison Summary

| Feature | STDIO | HTTP+SSE |
|---------|-------|----------|
| **Location** | Same machine | Any network |
| **Users** | Single | Multiple |
| **Auth** | Process permissions | OAuth/Tokens |
| **Setup** | Simple | Complex |
| **Latency** | ~0ms | Network dependent |
| **Scalability** | None | Horizontal |
| **Streaming** | stdout | SSE |
| **Debugging** | stderr logs | HTTP logs + Inspector |
| **Best For** | Local tools, dev | Shared services, cloud |

## 10. Key Takeaways

### HTTP+SSE Transport

1. **Architecture**: Client → POST requests, Server → SSE responses
2. **Sessions**: Managed via session_id in URL
3. **Auth**: OAuth 2.1 with scopes recommended
4. **Reconnection**: Implement exponential backoff
5. **Use Cases**: Remote access, multi-user, cloud deployment

### When to Use HTTP+SSE

| ✅ Use HTTP+SSE For | ❌ Don't Use HTTP+SSE For |
|--------------------|--------------------------|
| Shared team tools | Personal local tools |
| Cloud deployment | Development/testing |
| Multi-user access | Single-user apps |
| Enterprise services | Simple scripts |

### Next Steps

In **Notebook 5: Building Your First MCP Server**, we'll:
- Build a complete server with FastMCP
- Implement tools, resources, and prompts
- Add proper error handling and validation

## 11. Exercises

### Exercise 1: Add Health Check Endpoint

Extend the HTTP server with a `/health` endpoint that returns server status.

In [ ]:
# Exercise 1: Implement health check
HEALTH_CHECK = '''
async def health_handler(request: web.Request) -> web.Response:
    """Health check endpoint."""
    # TODO: Return JSON with:
    # - status: "healthy"
    # - version: server version
    # - active_sessions: count of active sessions
    # - uptime: server uptime
    pass
'''

print("Complete the health check handler:")
print(HEALTH_CHECK)

### Exercise 2: Implement Rate Limiting

Add rate limiting to prevent abuse of the HTTP server.

In [ ]:
# Exercise 2: Rate limiter
RATE_LIMITER = '''
from collections import defaultdict
from datetime import datetime, timedelta

class RateLimiter:
    """Simple rate limiter."""
    
    def __init__(self, max_requests: int, window_seconds: int):
        self.max_requests = max_requests
        self.window = timedelta(seconds=window_seconds)
        self.requests = defaultdict(list)  # client_id -> [timestamps]
    
    def is_allowed(self, client_id: str) -> bool:
        """Check if request is allowed."""
        # TODO: Implement rate limiting logic
        # 1. Remove old timestamps outside window
        # 2. Check if under limit
        # 3. Add current timestamp if allowed
        pass
'''

print("Implement the rate limiter:")
print(RATE_LIMITER)

### Exercise 3: Test Client Connection

Write a test script that:
1. Connects to an HTTP+SSE server
2. Sends initialize request
3. Lists tools
4. Calls a tool

In [ ]:
# Exercise 3: Test client script outline
TEST_CLIENT = '''
async def test_mcp_server(base_url: str):
    """Test MCP HTTP+SSE server."""
    client = MCPHttpClient(base_url)
    
    # TODO:
    # 1. Connect and get session
    # 2. Send initialize
    # 3. Wait for response
    # 4. Send initialized notification
    # 5. List tools
    # 6. Call a tool
    # 7. Print results
    pass
'''

print("Create the test client:")
print(TEST_CLIENT)

---

## References

- [MCP HTTP+SSE Transport Spec](https://modelcontextprotocol.io/docs/learn/transports#http-with-sse)
- [Server-Sent Events MDN](https://developer.mozilla.org/en-US/docs/Web/API/Server-sent_events)
- [OAuth 2.1 Draft](https://oauth.net/2.1/)
- [aiohttp Documentation](https://docs.aiohttp.org/)

---

**Next Notebook:** [05_Building_Your_First_MCP_Server.ipynb](./05_Building_Your_First_MCP_Server.ipynb)